# **Homework 1 - task 25**
The main goal of this homework is to transform existing evaluation datasets into a format suitable for evaluating the linguistic skills of Large Language Models (LLMs) by reframing tasks as multi-choice Question Answering (QA) tasks, providing effective prompts, and generating distractors where necessary, all formatted in JSON Lines standard for submission.

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# **1. Data Loading:**


First we need to download the datasets from evalita to our python eviroment



In [2]:
!unzip task-25-textentail-20240322T151548Z-001.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


## 1.1 visualize initial data

**dev.xml** is the training (development) dataset.


**test_gold.xml** is the test dataset.

In [3]:
def vizualize_original_data(file_path):
  # Open the file and read its contents
  with open(file_path, 'r') as file:
      count = 0
      print_line = False

      for line in file:
          if '<pair' in line:
              print_line = True

              count += 1

          if '</pair>' in line:
              print(line.strip())
              print_line = False

          if print_line:
              print(line.strip())

          if count == 3:
              break

# Check info in dev.xml
file_path = 'task-25-textentail/dev.xml'
vizualize_original_data(file_path)

<pair entailment="YES" id="1001" task="WIKI">
<t>'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.</t>
<h>'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.</h>
</pair>
<pair entailment="YES" id="1002" task="WIKI">
<t>Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.</t>
<h>Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.</h>
</pair>
<pair entailment="YES" id="1003" task="WIKI">


In [4]:
# Check info in test_gold.xml
file_path = 'task-25-textentail/test_gold.xml'
vizualize_original_data(file_path)

<pair id="1401" task="WIKI" entailment="NO">
<t>Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.</t>
<h>Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.</h>
</pair>
<pair id="1402" task="WIKI" entailment="NO">
<t>Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.</t>
<h>Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.</h>
</pair>
<pair id="1403" task="WIKI" entailment="YES">


## 1.2 Data Reframing:
  First we need to change the type of the file from xml to json. Then we want to convert that json into the following format:
```JSON
{
    "id":           int,
    "text":         str,
    "hypothesis":   str,
    "choices":      list[str],
    "label":        int
}
```

In [5]:
import importlib

# Check if the package is installed, and install it if it is not
def check_and_install(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"{package} is not installed. Installing...")
        import subprocess
        subprocess.check_call(["pip", "install", package])

check_and_install('xmltodict')


xmltodict is already installed.


In [6]:
import xmltodict
import json

# Define a function to convert an XML file to JSON Lines format
def xml_to_jsonl(xml_file, jsonl_file):
    with open(xml_file, 'r') as f:
        xml_data = f.read()

    # Parse XML into a dictionary
    data_dict = xmltodict.parse(xml_data)

    # Determine the root element of the XML and handle accordingly
    root_key = next(iter(data_dict))  # Get the first key in the dictionary
    root_element = data_dict[root_key]

    with open(jsonl_file, 'w') as f:
        # If the root element contains multiple elements, iterate over them
        if isinstance(root_element, list):
            for element in root_element:
                # Write each element as a separate line in JSON Lines format
                print(json.dumps(element, indent=4, sort_keys=True), file=f)
        else:
            # Write the single root element as a JSON object on a single line
            print(json.dumps(root_element, indent=4, sort_keys=True), file=f)



input_xml_path = 'task-25-textentail/dev.xml'
output_json_path = 'dev_out.jsonl'
xml_to_jsonl(input_xml_path, output_json_path)

input_xml_path = 'task-25-textentail/test_gold.xml'
output_json_path = 'test_gold_out.jsonl'
xml_to_jsonl(input_xml_path, output_json_path)

## 1.3 Visualize jsonl information

In [7]:
import json

def visualize_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    if 'pair' in data:
        pairs = data['pair'][:3]
        for pair in pairs:
            print("ID:", pair["@id"])
            print("Entailment:", pair["@entailment"])
            print("Task:", pair["@task"])
            print("T:", pair["t"])
            print("H:", pair["h"])
            print()

json_file = 'dev_out.jsonl'
visualize_data(json_file)


ID: 1001
Entailment: YES
Task: WIKI
T: 'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.
H: 'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.

ID: 1002
Entailment: YES
Task: WIKI
T: Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.
H: Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.

ID: 1003
Entailment: YES
Task: WIKI
T: Viene eletto sindaco di Catanzaro al ballottaggio del giugno del 2006, con il 50,8% dei voti.
H: Viene eletto sindaco di Catanzaro al ballottaggio del giugno 2006, con il 50,8% dei voti.



In [8]:
json_file = 'test_gold_out.jsonl'
visualize_data(json_file)

ID: 1401
Entailment: NO
Task: WIKI
T: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.
H: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.

ID: 1402
Entailment: NO
Task: WIKI
T: Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.
H: Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.

ID: 1403
Entailment: YES
Task: WIKI
T: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Como, nel 1967, riprese a giocare nelle fila del FC Chiasso nella Serie A Svizzera.
H: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Lecco, nel 1967, tornò a giocare nelle fila del FC Chiasso nella Serie A Svizzera.




## 1.4 Format jsonl

In order to get the json file with the desired format:

1. Rename @id to id.
2. Rename @entailment to label and change YES or NO to 0 or 1 respectivly
3. Rename @task to remove it since it's not present in the desired format.
4. Replace t with text.
5. Replace h with hypothesis.
6. Add a choices key with entailed or not entailed


In [9]:
import json

# Rearrange the JSON file to match the required format
def rearrange_json(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)

    for pair in data["pair"]:
        if '@id' in pair and 't' in pair and 'h' in pair and '@entailment' in pair:
            # Rearrange the keys
            new_pair = {
                "id": int(pair['@id']),
                "text": pair['t'],
                "hypothesis": pair['h'],
                "choices": ["entailed", "not entailed"],
                "label": 0 if pair['@entailment'] == "YES" else 1
            }
            pair.clear()  
            pair.update(new_pair)  

    with open(file_path, "w") as file:
        json.dump(data, file, indent=2)


file_path_test_gold = "test_gold_out.jsonl"
rearrange_json(file_path_test_gold)

file_path_dev = "dev_out.jsonl"
rearrange_json(file_path_dev)


## 1.5 Visualize final jsonl

In [10]:
import json

# Function to visualize the data in the JSON file
def visualize_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    if 'pair' in data:
        pairs = data['pair'][:3]
        for pair in pairs:
            print("id:", pair["id"])
            print("text:", pair["text"])
            print("hypothesis:", pair["hypothesis"])
            print("choices", pair["choices"])
            print("label:", pair["label"])
            print()

json_file = 'dev_out.jsonl'
visualize_data(json_file)


id: 1001
text: 'Pieralfonso Fratta Pasini' è un imprenditore e un politico italiano.
hypothesis: 'Pieralfonso Fratta Pasini' è un imprenditore e politico italiano.
choices ['entailed', 'not entailed']
label: 0

id: 1002
text: Del gruppo di Ravenna, facevano parte Alberto Acquacalda di Ravenna, Turiddu Candoli di Cervia, e Manoni.
hypothesis: Del gruppo di Ravenna, facevano parte Turiddu Candoli di Cervia, Acquacalda e Manoni.
choices ['entailed', 'not entailed']
label: 0

id: 1003
text: Viene eletto sindaco di Catanzaro al ballottaggio del giugno del 2006, con il 50,8% dei voti.
hypothesis: Viene eletto sindaco di Catanzaro al ballottaggio del giugno 2006, con il 50,8% dei voti.
choices ['entailed', 'not entailed']
label: 0



In [11]:
json_file = 'test_gold_out.jsonl'
visualize_data(json_file)

id: 1401
text: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti.
hypothesis: Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.
choices ['entailed', 'not entailed']
label: 1

id: 1402
text: Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto.
hypothesis: Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.
choices ['entailed', 'not entailed']
label: 1

id: 1403
text: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Como, nel 1967, riprese a giocare nelle fila del FC Chiasso nella Serie A Svizzera.
hypothesis: Bruschini però non si perse d'animo e, tornato sulle rive del lago di Lecco, nel 1967, tornò a giocare nelle fila del FC Chiasso nella Serie A Svizzera.
choices ['entailed', 'not entailed']
label: 0



## **2. Promt formulation**

In order to use this dataset we need to generate three prompts that can be used to get if a text is entailed to a hypothesis and then insert them into a json file. The three prompts are:

**Template 1:**


Prompt: "La frase '{text}' sostiene la frase '{hypothesis}'?"

Translation: "Does the statement '{text}' entail the statement '{hypothesis}'?"4

**Template 2:**


Prompt: "La frase '{text}' implica la frase '{hypothesis}'?"

Translation: "Does the statement '{text}' imply the statement '{hypothesis}'?"

**Template 3:**


Prompt: "La frase '{hypothesis}' può essere dedotta dalla frase '{text}'?"

Translation: "Can the statement '{hypothesis}' be deduced from the statement '{text}'?"


In [12]:
import json

# Function to generate a JSON Lines file with a list of prompts
def generate_json(prompts, output_file):
    data = []
    for prompt in prompts:
        data.append({"prompt": prompt})

    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    with open(output_file, 'r', encoding='utf-8') as json_file:
        print(json_file.read())


prompts = [
    "La frase {{text}} sostiene la frase {{hypothesis}}?",
    "La frase {{text}} implica la frase {{hypothesis}}?",
    "La frase {{hypothesis}} può essere dedotta dalla frase {{text}}?"
]

output_file = "prompts.jsonl"
generate_json(prompts, output_file)
print(f"JSON file '{output_file}' generated successfully with {len(prompts)} prompts.")



[
    {
        "prompt": "La frase {{text}} sostiene la frase {{hypothesis}}?"
    },
    {
        "prompt": "La frase {{text}} implica la frase {{hypothesis}}?"
    },
    {
        "prompt": "La frase {{hypothesis}} pu\u00f2 essere dedotta dalla frase {{text}}?"
    }
]
JSON file 'prompts.jsonl' generated successfully with 3 prompts.


# **3. Llama 2 set up**

In [13]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [14]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or A100 to have sufficient RAM.

## 3.1 Install

In [15]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [16]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

ERROR: Invalid requirement: '#'


In [17]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

##3.2 Load Model


In [18]:
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

c:\Users\35193\miniconda3\envs\gpukernel\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **4. Evaluate Homework prompts**

## 4.1. Test response of Promtps

In [19]:
# Load your JSON file and process each pair
with open('test_gold_out.jsonl', 'r') as file:
    data = json.load(file)
    if 'pair' in data:
        pairs = data['pair'][:4]
        for pair in pairs:
            id = pair["id"]
            text = pair["text"]
            hypothesis = pair["hypothesis"]
            choices = pair["choices"]
            label = pair["label"]


            # Format the prompt with text and hypothesis
            prompt_formatted = f"La frase {text} sostiene la frase {hypothesis}?"

            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            print(f"Probabilities: {probabilities}")
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]
            prediction_confidence = probabilities[prediction_index] * 100
            prediction = {"label": prediction_label, "confidence": round(prediction_confidence, 1)}

            # Compare prediction with actual label
            is_correct = prediction_label == choices[label]

            # Visualize the formatted prompt and the prediction details
            print("Prompt:", prompt_formatted)
            print("Prediction:", prediction)
            print("Actual Label:", choices[label])
            print("Correct Prediction:", is_correct)
            print()


Probabilities: [0.5765582  0.32311448 0.10032736]
Prompt: La frase Il 15 febbraio 2009 viene chiamato ad allenare il grosseto, dopo l'esonero di Gustinetti. sostiene la frase Il 15 febbraio 2009 viene chiamato ad allenare il grosseto calcio, dopo l'esonero di Gustinetti.?
Prediction: {'label': 'entailed', 'confidence': 57.7}
Actual Label: not entailed
Correct Prediction: False

Probabilities: [0.13443957 0.8531037  0.01245672]
Prompt: La frase Il 9 giugno 2009 Cirillo rescinde il contratto con la società amaranto. sostiene la frase Il 9 giugno 2009 Cirillo rescinde consensualmente il proprio contratto con la società amaranto.?
Prediction: {'label': 'not entailed', 'confidence': 85.3}
Actual Label: not entailed
Correct Prediction: True

Probabilities: [0.00485643 0.0193718  0.9757717 ]
Prompt: La frase Bruschini però non si perse d'animo e, tornato sulle rive del lago di Como, nel 1967, riprese a giocare nelle fila del FC Chiasso nella Serie A Svizzera. sostiene la frase Bruschini però 

##4.2. Compare the diferent prompts

In [20]:
import json

total_correct = 0
total_pairs = 0

# Load the JSONL file and process each pair
with open('test_gold_out.jsonl', 'r') as file:
    data = json.load(file)
    if 'pair' in data:
        pairs = data['pair']
        for pair in pairs:
            id = pair["id"]
            text = pair["text"]
            hypothesis = pair["hypothesis"]
            choices = pair["choices"]
            label = pair["label"]

            # Format the prompt with actual text and hypothesis
            prompt_formatted = f"La frase {hypothesis} può essere dedotta dalla frase {text}?"

            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]

            # Compare prediction with actual label
            is_correct = prediction_label == choices[label]

            # Accumulate statistics
            total_correct += is_correct
            total_pairs += 1

# Calculate overall statistics only if at least one pair was processed
if total_pairs > 0:
    accuracy = total_correct / total_pairs * 100

    # Print statistics
    print("Overall Statistics:")
    print("Total Pairs:", total_pairs)
    print("Total Correct:", total_correct)
    print("Accuracy:", round(accuracy, 2), "%")
else:
    print("No pairs found in the JSONL file.")


Overall Statistics:
Total Pairs: 400
Total Correct: 227
Accuracy: 56.75 %


In [21]:
import json
import torch

def prompt_testing(prompt):
    total_correct = 0
    total_pairs = 0
    total_confidence = 0

    # Load the JSONL file and process each pair
    with open('test_gold_out.jsonl', 'r') as file:
        data = json.load(file)
        if 'pair' in data:
            pairs = data['pair']
            for pair in pairs:
                id = pair["id"]
                text = pair["text"]
                hypothesis = pair["hypothesis"]
                choices = pair["choices"]
                label = pair["label"]
                
                # Format with text and hypothesis 
                prompt_formatted = prompt.replace('{{text}}', text).replace('{{hypothesis}}', hypothesis)
                input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
                output = model_llama(**input)
                logits = output.logits[0].detach().cpu().numpy()

                # Obtain the predicted label directly from logits
                probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
                prediction_index = 1 if probabilities[1] > probabilities[0] else 0
                prediction_label = choices[prediction_index]

                # Compare prediction with actual label
                is_correct = prediction_label == choices[label]

                # Accumulate statistics
                total_correct += is_correct
                total_pairs += 1
                total_confidence += probabilities[prediction_index]

    # Calculate statistics 
    if total_pairs > 0:
        accuracy = total_correct / total_pairs * 100
        mean_confidence = total_confidence / total_pairs

        # Print statistics
        print("Total Pairs:", total_pairs)
        print("Total Correct:", total_correct)
        print("Accuracy:", round(accuracy, 2), "%")
        print("Mean Confidence:", round(mean_confidence, 2))
    else:
        print("No pairs found in the JSONL file.")

# Load prompts from the JSONL file and test each prompt
with open('prompts.jsonl', 'r') as prompts_file:
    prompt_data = json.load(prompts_file)
    for prompt_item in prompt_data:
        prompt = prompt_item["prompt"]
        print("\nPrompt:", prompt)
        prompt_testing(prompt)



Prompt: La frase {{text}} sostiene la frase {{hypothesis}}?
Total Pairs: 400
Total Correct: 291
Accuracy: 72.75 %
Mean Confidence: 0.59

Prompt: La frase {{text}} implica la frase {{hypothesis}}?
Total Pairs: 400
Total Correct: 292
Accuracy: 73.0 %
Mean Confidence: 0.69

Prompt: La frase {{hypothesis}} può essere dedotta dalla frase {{text}}?
Total Pairs: 400
Total Correct: 227
Accuracy: 56.75 %
Mean Confidence: 0.6
